## ** Also refer to wikipedia_scraper_RESTART where I continued working after I found blank paragraphs for many US Leaders in the JSON output. **
At first I started updating in this file, but this became too complicated having to comment out later code to stop from running.

# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## 0. Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

-- I did not rename the venv created to the name above and continued working.

Activate it and add it to you `.gitignore` file. 

You will find more info about virtual environments in the course content and on the web.

## 1. API Scraping

### 1a. A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword. NOTE: external libraries need to be installed first. Check the [request Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the documentation to:

1. Use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. Check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is an explanation of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [59]:
# import the requests library (1 line)
import requests

# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
root_url = "https://country-leaders.onrender.com"

# assign the /status endpoint to another variable called status_url (1 line)
status_url = root_url + "/status"

# query the /status endpoint using the get() method and store it in the req variable (1 line)
response = requests.get(status_url)

# check the status_code using a condition and print appropriate messages (4 lines)
if response.status_code == 200:
    print(response.text)
else:
    print("Error:", response.status_code)

"Alive"


### 1b. Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. 


In [60]:
# Set the countries_url variable (1 line)
countries_url = root_url + "/countries"

# query the /countries endpoint using the get() method and store it in the req variable (1 line)
response = requests.get(countries_url)

# Get the JSON content and store it in the countries variable (1 line)
countries = response.json()

# display the request's status code and the countries variable (1 line)
print(f"Status: {response.status_code}, Countries: {list(countries.keys())}")

Status: 403, Countries: ['message']


### 1c. Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [61]:
# Set the cookie_url variable (1 line)
cookie_url = root_url + "/cookie"

# Query the endpoint, set the cookies variable and display it (2 lines)
cookies = requests.get(cookie_url).cookies
print(cookies)

<RequestsCookieJar[<Cookie user_cookie=d3a22560-83af-42ce-a5b0-aa51910290d9 for country-leaders.onrender.com/>]>


Try to query the countries endpoint using the cookie, save the output and print it.

In [62]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
countries = requests.get(countries_url, cookies=cookies).json()

# display the countries variable (1 line)
print(countries)

['be', 'us', 'ru', 'fr', 'ma']


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### 1d. Getting the actual data from the API

Query the `/leaders` endpoint.

In [63]:
# Set the leaders_url variable (1 line)
leaders_url = root_url + "/leaders"

# query the /leaders endpoint, assign the output to the leaders variable (1 line)
leaders = requests.get(leaders_url, cookies=cookies).json()

# display the leaders variable (1 line)
print(leaders)

{'message': 'Please specify a country'}


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [64]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)
# assign the output to the leaders variable (1 line)
leaders = requests.get(leaders_url, cookies=cookies, params={"country": countries[0]}).json()

# display the leaders variable (1 line)
print(leaders)

[{'id': 'Q12978', 'first_name': 'Guy', 'last_name': 'Verhofstadt', 'birth_date': '1953-04-11', 'death_date': None, 'place_of_birth': 'Dendermonde', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'start_mandate': '1999-07-12', 'end_mandate': '2008-03-20'}, {'id': 'Q12981', 'first_name': 'Yves', 'last_name': 'Leterme', 'birth_date': '1960-10-06', 'death_date': None, 'place_of_birth': 'Wervik', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme', 'start_mandate': '2009-11-25', 'end_mandate': '2011-12-06'}, {'id': 'Q12983', 'first_name': 'Herman', 'last_name': 'None', 'birth_date': '1947-10-31', 'death_date': None, 'place_of_birth': 'Etterbeek', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy', 'start_mandate': '2008-12-30', 'end_mandate': '2009-11-25'}, {'id': 'Q14989', 'first_name': 'Léon', 'last_name': 'Delacroix', 'birth_date': '1867-12-27', 'death_date': '1929-10-15', 'place_of_birth': 'Saint-Josse-ten-Noode', 'wikipedia_url': 'https://nl

### 1e. A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [65]:
# 4 lines
leaders_per_country = {}
for country_code in countries:
    leaders_per_country[country_code] = requests.get(leaders_url, cookies=cookies, params={"country": country_code}).json()
print(leaders_per_country)

{'be': [{'id': 'Q12978', 'first_name': 'Guy', 'last_name': 'Verhofstadt', 'birth_date': '1953-04-11', 'death_date': None, 'place_of_birth': 'Dendermonde', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'start_mandate': '1999-07-12', 'end_mandate': '2008-03-20'}, {'id': 'Q12981', 'first_name': 'Yves', 'last_name': 'Leterme', 'birth_date': '1960-10-06', 'death_date': None, 'place_of_birth': 'Wervik', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme', 'start_mandate': '2009-11-25', 'end_mandate': '2011-12-06'}, {'id': 'Q12983', 'first_name': 'Herman', 'last_name': 'None', 'birth_date': '1947-10-31', 'death_date': None, 'place_of_birth': 'Etterbeek', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy', 'start_mandate': '2008-12-30', 'end_mandate': '2009-11-25'}, {'id': 'Q14989', 'first_name': 'Léon', 'last_name': 'Delacroix', 'birth_date': '1867-12-27', 'death_date': '1929-10-15', 'place_of_birth': 'Saint-Josse-ten-Noode', 'wikipedia_url': 'htt

In [66]:
# or 1 line (I actually have 2 lines, if you include the print statement)
# leaders_per_country = {code: requests.get(leaders_url, cookies=cookies, params={"country": code}).json() for code in countries}
# print(leaders_per_country)

# Forcing 1 line
print({code: requests.get(leaders_url, cookies=cookies, params={"country": code}).json() for code in countries})

{'be': [{'id': 'Q12978', 'first_name': 'Guy', 'last_name': 'Verhofstadt', 'birth_date': '1953-04-11', 'death_date': None, 'place_of_birth': 'Dendermonde', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'start_mandate': '1999-07-12', 'end_mandate': '2008-03-20'}, {'id': 'Q12981', 'first_name': 'Yves', 'last_name': 'Leterme', 'birth_date': '1960-10-06', 'death_date': None, 'place_of_birth': 'Wervik', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme', 'start_mandate': '2009-11-25', 'end_mandate': '2011-12-06'}, {'id': 'Q12983', 'first_name': 'Herman', 'last_name': 'None', 'birth_date': '1947-10-31', 'death_date': None, 'place_of_birth': 'Etterbeek', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy', 'start_mandate': '2008-12-30', 'end_mandate': '2009-11-25'}, {'id': 'Q14989', 'first_name': 'Léon', 'last_name': 'Delacroix', 'birth_date': '1867-12-27', 'death_date': '1929-10-15', 'place_of_birth': 'Saint-Josse-ten-Noode', 'wikipedia_url': 'htt

It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [67]:
# < 15 lines
import requests

def get_leaders():
    root_url = "https://country-leaders.onrender.com"
    cookies = requests.get(root_url + "/cookie").cookies
    countries = requests.get(root_url + "/countries", cookies=cookies).json()
    leaders_url = root_url + "/leaders"
    leaders_per_country = {c: requests.get(leaders_url, cookies=cookies, params={"country": c}).json() for c in countries}
    return leaders_per_country

Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [ ]:
# 2 lines
# leaders_per_country = get_leaders()
# print(leaders_per_country)

{'be': [{'id': 'Q12978', 'first_name': 'Guy', 'last_name': 'Verhofstadt', 'birth_date': '1953-04-11', 'death_date': None, 'place_of_birth': 'Dendermonde', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'start_mandate': '1999-07-12', 'end_mandate': '2008-03-20'}, {'id': 'Q12981', 'first_name': 'Yves', 'last_name': 'Leterme', 'birth_date': '1960-10-06', 'death_date': None, 'place_of_birth': 'Wervik', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme', 'start_mandate': '2009-11-25', 'end_mandate': '2011-12-06'}, {'id': 'Q12983', 'first_name': 'Herman', 'last_name': 'None', 'birth_date': '1947-10-31', 'death_date': None, 'place_of_birth': 'Etterbeek', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy', 'start_mandate': '2008-12-30', 'end_mandate': '2009-11-25'}, {'id': 'Q14989', 'first_name': 'Léon', 'last_name': 'Delacroix', 'birth_date': '1867-12-27', 'death_date': '1929-10-15', 'place_of_birth': 'Saint-Josse-ten-Noode', 'wikipedia_url': 'htt

## 2. Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [68]:
# 3 lines
wiki_url = list(leaders_per_country.values())[1][1]['wikipedia_url']
   # define header with user-agent & respect robots policy
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/26.0.1 Safari/605.1.15 wikipedia-scraper/1.0 (email: kristin.nuyens@gmail.com)"}
response = requests.get(wiki_url, headers=headers)
print(response.text)

KeyboardInterrupt: 

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [57]:
# 3 lines
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Barack Obama - Wikipedia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-en

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [58]:
# 2 lines
paragraphs = soup.select("div.mw-parser-output > p")
paragraph_texts = [p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True)]

#test
print(paragraph_texts)

[]


If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one:
   * Store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
   * Exit the loop

In [49]:
# <10 lines
# Select all paragraphs in the main content area
paragraphs = soup.select("div.mw-parser-output > p")

first_paragraph = None
for p in paragraphs:
    text = p.get_text(strip=True)
    # Skip empty or non-word paragraphs
    if not text or not any(ch.isalpha() for ch in text):
        continue
    first_paragraph = text
    break

if first_paragraph:
    print(first_paragraph)
# else:
    # print("No valid paragraph found.")



At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [52]:
# 10 lines
# def get_first_paragraph(wikipedia_url):
#   print(wikipedia_url) # keep this for the rest of the notebook
def get_first_paragraph(wikipedia_url):
    print(wikipedia_url)  # keep this for the rest of the notebook
    headers = {"User-Agent": "wikipedia-scraper/1.0 (kristin.nuyens@gmail.com)"}
    response = requests.get(wikipedia_url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    paragraphs = soup.select("div.mw-parser-output > p")

    print("Number of paragraphs found:", len(paragraphs))
#   [insert your code]
#   return first_paragraph
    first_paragraph = None
    for p in paragraphs:
        if p.get_text(strip=True):
            first_paragraph = p.get_text(strip=True)
            # starts_with_bold = p.find("b") is not None or p.find("strong") is not None - seems this is not needed (and 1 extra li)
            break
    return first_paragraph

In [53]:
# Test: 3 lines
paragraph = get_first_paragraph("https://en.wikipedia.org/wiki/Angela_Merkel")
print(paragraph)

# double check it is refreshing the URL data as I had trouble with this:
paragraph = get_first_paragraph("https://en.wikipedia.org/wiki/Barack_Obama")
print(paragraph)
paragraph = get_first_paragraph("https://en.wikipedia.org/wiki/Donald_Trump")
print(paragraph)

https://en.wikipedia.org/wiki/Angela_Merkel
Number of paragraphs found: 6
Angela Dorothea Merkel(German pronunciation:[aŋˈɡeːladoʁoˈteːaˈmɛʁkl̩]ⓘ;[a]néeKasner; born 17 July 1954) is a German retired politician who served asChancellor of Germanyfrom 2005 to 2021. She is theonly womanto have held the office and the only from formerEast Germany. She wasLeader of the Oppositionfrom 2002 to 2005 andLeader of the Christian Democratic Union(CDU) from 2000 to 2018.[9]
https://en.wikipedia.org/wiki/Barack_Obama
Number of paragraphs found: 0
None
https://en.wikipedia.org/wiki/Donald_Trump
Number of paragraphs found: 7
Donald John Trump(born June 14, 1946) is an American politician, media personality, and businessman who is the 47thpresident of the United States. A member of theRepublican Party, he served as the 45th president from 2017 to 2021.


### 2a. Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [ ]:
# 3 lines
import re

def clean_paragraph(text):
    return re.sub(r"\s+", " ", re.sub(r"\[[^\]]*\]|\([^\)]*?pronunciation[^\)]*?\)|&[a-z]+?;", "", text or "")).strip()

Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [ ]:
# 10 lines
import re
from bs4 import BeautifulSoup

def get_first_paragraph(wikipedia_url, session):
    """Fetch the first paragraph from a Wikipedia page and clean it."""
    response = session.get(wikipedia_url)
    soup = BeautifulSoup(response.text, "html.parser")
    for p in soup.select("div.mw-parser-output > p"):
        text = p.get_text(strip=True)
        if text:
            # Clean the paragraph using regex before returning
            return re.sub(r"\s+", " ", re.sub(r"\[[^\]]*\]|\([^\)]*?pronunciation[^\)]*?\)|&[a-z]+?;", "", text)).strip()
    return None


Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [ ]:
# < 20 lines


## 3. Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [ ]:
# < 20 lines
from bs4 import BeautifulSoup
import requests
import time

USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/26.0.1 Safari/605.1.15 wikipedia-scraper/1.0 (email: kristin.nuyens@gmail.com)"
HEADERS = {"User-Agent": USER_AGENT}
ROOT = "https://country-leaders.onrender.com"

def get_cookies():
    return requests.get(ROOT + "/cookie", headers=HEADERS).cookies

def get_first_paragraph(wikipedia_url):
    response = requests.get(wikipedia_url, headers=HEADERS)
    soup = BeautifulSoup(response.text, "html.parser")
    for p in soup.select("div.mw-parser-output > p"):
        text = p.get_text(strip=True)
        if text:
            return text
    return None

def get_leaders():
    cookies = get_cookies()
    countries = requests.get(ROOT + "/countries", cookies=cookies, headers=HEADERS).json()
    leaders_per_country = {}
    for c in countries:
        # print(f"Getting leaders for {c}") # this code is just used to show the progress made in the  output
        # retry up to 2 times if not a valid list
        for _ in range(2):
            data = requests.get(ROOT + "/leaders", cookies=cookies, headers=HEADERS, params={"country": c}).json()
            if isinstance(data, list):
                break
            # print(f"Warning: got {type(data)} for {c}, refreshing cookie...") # this is used to ID if list or dictionary
            cookies = get_cookies()
            time.sleep(0.5)

        # if not isinstance(data, list):
        #     print(f"Skipping {c} (invalid response: {data})") # this was used to help troubleshooting list vs dictionary
        #     continue

        for l in data:
            if "wikipedia_url" in l:
                # print(l["wikipedia_url"]) # this was used to verify progress
                l["first_paragraph"] = get_first_paragraph(l["wikipedia_url"])
            # else:
            #     print(f"Missing wikipedia_url for {l}") # more troubloshooting
        leaders_per_country[c] = data
        time.sleep(0.5)  # polite delay between countries
    return leaders_per_country

In [ ]:
# Check the output of your function (2 lines)
# leaders_per_country = get_leaders()
# print(list(leaders_per_country.keys()))

Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [ ]:
# < 25 lines
# See above - have continued updating there...

Check the output of your function again.

In [ ]:
# Check the output of your function (1 line)
# see above

Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [ ]:
# < 20 lines
from bs4 import BeautifulSoup
import requests
import time

USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/26.0.1 Safari/605.1.15 wikipedia-scraper/1.0 (email: kristin.nuyens@gmail.com)"
HEADERS = {"User-Agent": USER_AGENT}
ROOT = "https://country-leaders.onrender.com"

def get_cookies(session):
    return session.get(ROOT + "/cookie").cookies

def get_first_paragraph(wikipedia_url, session):
    response = session.get(wikipedia_url)
    soup = BeautifulSoup(response.text, "html.parser")
    for p in soup.select("div.mw-parser-output > p"):
        text = p.get_text(strip=True)
        if text:
            return text
    return None

def get_leaders():
    leaders_per_country = {}
    with requests.Session() as session:
        session.headers.update(HEADERS)
        # Refresh cookie for /countries call
        cookies = get_cookies(session)
        countries = session.get(ROOT + "/countries", cookies=cookies).json()
        for country in countries:
            # Refresh cookie for this country
            cookies = get_cookies(session)
            response = session.get(ROOT + "/leaders", cookies=cookies, params={"country": country})
            data = response.json()
            # Handle dict responses
            if isinstance(data, dict):
                data = data.get("leaders", [])
            # Fetch first paragraph for each leader
            for leader in data:
                if "wikipedia_url" in leader:
                    leader["first_paragraph"] = get_first_paragraph(leader["wikipedia_url"], session)
            leaders_per_country[country] = data
    return leaders_per_country

leaders_per_country = get_leaders()

Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [ ]:
# <25 lines
# I already incorporated above

Test your new functions.



In [ ]:
# # --- FULL FUNCTION TEST WITH FIRST + LAST COUNTRY/LEADER ---
# import requests

# print("1️⃣ Creating session...")
# with requests.Session() as session:
#     session.headers.update(HEADERS)
#     print("2️⃣ Fetching countries with fresh cookie...")
#     cookies = get_cookies(session)
#     countries_response = session.get(ROOT + "/countries", cookies=cookies)
#     countries = countries_response.json()
#     print("Countries:", countries)
#     print("-" * 60)

#     # --- First country / first leader ---
#     first_country = countries[0]
#     cookies = get_cookies(session)
#     leaders_response = session.get(ROOT + "/leaders", cookies=cookies, params={"country": first_country})
#     leaders = leaders_response.json()
#     first_leader = leaders[0] if isinstance(leaders, list) else list(leaders.values())[0]
#     print(f"\nFirst country: {first_country}")
#     print("First leader info:", first_leader.get("first_name", ""), first_leader.get("last_name", ""))
#     if "wikipedia_url" in first_leader:
#         paragraph = get_first_paragraph(first_leader["wikipedia_url"], session)
#         print("First paragraph:", paragraph[:300], "...\n")

#     # --- Last country / last leader ---
#     last_country = countries[-1]
#     cookies = get_cookies(session)
#     leaders_response = session.get(ROOT + "/leaders", cookies=cookies, params={"country": last_country})
#     leaders = leaders_response.json()
#     last_leader = leaders[-1] if isinstance(leaders, list) else list(leaders.values())[-1]
#     print(f"\nLast country: {last_country}")
#     print("Last leader info:", last_leader.get("first_name", ""), last_leader.get("last_name", ""))
#     if "wikipedia_url" in last_leader:
#         paragraph = get_first_paragraph(last_leader["wikipedia_url"], session)
#         print("First paragraph:", paragraph[:300], "...\n")

#     # --- Full get_leaders() test ---
#     print("3️⃣ Testing get_leaders() full run (this may take a while ... < 4 mins)")
#     leaders_per_country = get_leaders()
#     print("Countries retrieved:", list(leaders_per_country.keys()))

## 4. Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [ ]:
# 3 lines
import json

# Save the dictionary to a JSON file
with open("leaders.json", "w", encoding="utf-8") as f:
    json.dump(leaders_per_country, f, ensure_ascii=False, indent=4)

# print("Saved leaders_per_country to leaders.json")

Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [ ]:
# 3 lines
import json

loaded_leaders = json.load(open("leaders.json", "r", encoding="utf-8"))
print("Success: loaded data matches the original!" if loaded_leaders == leaders_per_country else "Mismatch: loaded data differs.")

Make a function `save(leaders_per_country)` to call this code easily.

In [ ]:
# 3 lines
import json

def save(leaders_per_country):
    # Save leaders_per_country to leaders.json and confirm success.
    with open("leaders.json", "w", encoding="utf-8") as f:
        json.dump(leaders_per_country, f, ensure_ascii=False, indent=4)
    # print("Saved leaders_per_country to leaders.json")

In [ ]:
# Call the function (1 line)
save(leaders_per_country)

## 5. Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## (Optional) To go further

If you want to practice scraping, you can read this section and tackle the exercises.

1. Restructure your code by using OOP (see ReadMe).
2. You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!

# ** I started looking into using more OOP, but my output again returned missing paragraphs, so with limited time left I stopped looking into that.